In [1]:
import pandas as pd
import pyodbc as db
import db_config as config
import numpy as np
import time

#### Connect to the Database

In [2]:
conn = db.connect(DRIVER='SQL Server',
                 SERVER = config.server_name,
                 UID = config.user,
                 PWD=config.pwd,
                 DATABASE=config.database_name)

In [3]:
conn

#### Read the Tags from the excel sheet and modify data

In [4]:
monitor = pd.read_csv('RegisterData.csv')
monitor2= monitor.dropna(subset='Address').reset_index().drop(columns='index')
monitor2['Address'] = [int(i) for i in monitor2['Address']]

In [5]:
cols = monitor2.columns.tolist()
for col in cols:
    monitor2[col] = [str(i) for i in monitor2[col]]

In [6]:
monitor2

,Address,Register name,Scale,Description
0,103,PumpPowerON,nan,Power State - 0 or 1
1,207,ActualSetpointManualHI,0.1 ml/h,"The actual setpoint used in operating mode ""Ma..."
2,208,ActualSetpointManualLO,nan,Can be set via SetpointManual (register 00106-...
3,209,ActualPulseVolumeHI,1 nl,The actual pulse volume used in operating mode...
4,210,ActualPulseVolumeLO,nan,Can be set via SetPulseVolume (register 00108-...
5,211,ActualBatchDosingVolumeHI,0.001 ml,Can be set via SetBatchDosingVolume (register ...
6,212,ActualBatchDosingVolumeLO,nan,HMI.
7,213,ActualBatchDosingTimeHI,0.1 s,The actual batch dosing time used in operating...
8,214,ActualBatchDosingTimeLO,nan,Can be set via SetBatchDosingTime (register 00...
9,215,ActualPressureMax,0.1 bar,Actual value of (relative) pressure alarm limi...


In [7]:
monitor2 = monitor2.replace('nan', 'NA')

In [8]:
cols

['Address', 'Register name', 'Scale', 'Description']

#### Custom functions to create table, Write, read and delete tags

In [4]:
def createTable(db_name, tablename, columns, conn):
    insertCMD = f'''CREATE TABLE "{tablename}" ({columns});'''
    cur = conn.cursor()
    try:
        cur.execute(insertCMD)
        conn.commit()
        print(f"Table {tablename} was created in DB {db_name}")
    except(Exception, db.DatabaseError) as error:
        print(error)

In [5]:
def writeValues(metrics, conn, table):
    try:
        cur = conn.cursor()
    except (Exception, db.DatabaseError) as error:
        print(error)
    keys = list(metrics.keys())
    values = tuple(metrics.values())
    cols = '"' + ('","').join(keys) + '"'
    s_lens = "?,"*len(keys)
    s = s_lens.split(",")
    s = (",").join(s[:-1])
    insertQ = f""" INSERT INTO {table} ({cols})
                    VALUES({s})"""
    try:
        cur.execute(insertQ, values)
        conn.commit()
        print(f'Values Inserted: {values}')
    except (Exception, db.DatabaseError) as error:
        print(error)

In [6]:
def getData(tablename, orderby, conn):
    qu = f'select * from "{tablename}" order by "{orderby}" desc'
    alldata = pd.read_sql_query(qu, conn)
    return alldata

In [7]:
def delData(tablename, conn):
    try:
        cur = conn.cursor()
        q = f"delete from {tablename};"
        cur.execute(q)
        l = getData(tablename, conn)
        if len(l['Address'].tolist()) == 0:
            print("Delete Succesful")
    except (Exception, db.DatabaseError) as error:
        print(error)

In [8]:
def addColumns(tablename, columns, conn):
    try:
        cur = conn.cursor()
        q = f'''ALTER TABLE {tablename}
            ADD {columns};'''
        cur.execute(q)
        conn.commit()
        print(f'Columns Added')
    except (Exception, db.DatabaseError) as error:
        print(error)

In [9]:
def changeColumns(tablename, columns, conn):
    try:
        cur = conn.cursor()
        q = f'''ALTER TABLE {tablename}
            DROP COLUMN "{columns.split(' ')[0]}";'''
        cur.execute(q)
        conn.commit()
        addColumns(tablename, columns, conn)
        print(f'Columns changed')
    except (Exception, db.DatabaseError) as error:
        print(error)

#### Create the Data table

In [13]:
columns1 = ""
for col in monitor2.columns.tolist():
    if columns1 == "":
        columns1 = f'"{col}" text'
    else:
        columns1 = f'{columns1}, "{col}" text'

In [14]:
columns1

'"Address" text, "Register name" text, "Scale" text, "Description" text'

In [15]:
columns2 = ""
for col in monitor2['Address'].tolist():
    if columns2 == "":
        columns2 = f'"{col}" text'
    else:
        columns2 = f'{columns2}, "{col}" text'

In [16]:
columns2

'"103" text, "207" text, "208" text, "209" text, "210" text, "211" text, "212" text, "213" text, "214" text, "215" text, "216" text, "217" text, "218" text, "219" text, "301" text, "302" text, "303" text, "304" text, "305" text, "306" text, "307" text, "308" text, "309" text, "310" text, "311" text, "312" text, "313" text, "314" text, "315" text, "320" text, "321" text, "322" text, "323" text, "324" text, "325" text, "326" text, "327" text, "328" text'

In [17]:
createTable(config.database_name,'PoC_SP_MonitoringTags',columns1,conn)

Table PoC_SP_MonitoringTags was created in DB budig-bb-pltsql-05-d


In [19]:
createTable(config.database_name, 'PoC_SP_Metrics', columns2, conn)

Table PoC_SP_Metrics was created in DB budig-bb-pltsql-05-d


#### Write the tag info and test if all info is available

In [20]:
data = monitor2.to_dict('records')
for metrics in data:
    try:
        writeValues(metrics, conn, "PoC_SP_MonitoringTags")
    except pyodbc.Error as pe:
        print("Error:", pe)
        if pe.args[0] == "08S01":  # Communication error.
            # Nuke the connection and retry.
            try:
                conn.close()
                print("Connection Ended")
            except:
                pass
            continue

Values Inserted: ('103', 'PumpPowerON', 'NA', 'Power State - 0 or 1')
Values Inserted: ('207', 'ActualSetpointManualHI', '0.1 ml/h', 'The actual setpoint used in operating mode "Manual".')
Values Inserted: ('208', 'ActualSetpointManualLO', 'NA', 'Can be set via SetpointManual (register 00106-00107) or via the pump HMI.')
Values Inserted: ('209', 'ActualPulseVolumeHI', '1 nl', 'The actual pulse volume used in operating mode "Pulse".')
Values Inserted: ('210', 'ActualPulseVolumeLO', 'NA', 'Can be set via SetPulseVolume (register 00108-00109) or via the pump HMI.')
Values Inserted: ('211', 'ActualBatchDosingVolumeHI', '0.001 ml', 'Can be set via SetBatchDosingVolume (register 00110-00111) or via the pump')
Values Inserted: ('212', 'ActualBatchDosingVolumeLO', 'NA', 'HMI.')
Values Inserted: ('213', 'ActualBatchDosingTimeHI', '0.1 s', 'The actual batch dosing time used in operating mode "Batch".')
Values Inserted: ('214', 'ActualBatchDosingTimeLO', 'NA', 'Can be set via SetBatchDosingTime (

In [22]:
getData("PoC_SP_MonitoringTags", conn)

C:\Users\ramachandran\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Address,Register name,Scale,Description
0,103,PumpPowerON,NA,Power State - 0 or 1
1,207,ActualSetpointManualHI,0.1 ml/h,"The actual setpoint used in operating mode ""Ma..."
2,208,ActualSetpointManualLO,NA,Can be set via SetpointManual (register 00106-...
3,209,ActualPulseVolumeHI,1 nl,The actual pulse volume used in operating mode...
4,210,ActualPulseVolumeLO,NA,Can be set via SetPulseVolume (register 00108-...
5,211,ActualBatchDosingVolumeHI,0.001 ml,Can be set via SetBatchDosingVolume (register ...
6,212,ActualBatchDosingVolumeLO,NA,HMI.
7,213,ActualBatchDosingTimeHI,0.1 s,The actual batch dosing time used in operating...
8,214,ActualBatchDosingTimeLO,NA,Can be set via SetBatchDosingTime (register 00...
9,215,ActualPressureMax,0.1 bar,Actual value of (relative) pressure alarm limi...


In [23]:
Logs = pd.read_csv('user_logs.csv')

In [25]:
columns3 = ""
for col in Logs.columns.tolist():
    if columns3 == "":
        columns3 = f'"{col}" text'
    else:
        columns3 = f'{columns3}, "{col}" text'

In [26]:
createTable(config.database_name, 'PoC_SP_UserLogs', columns3, conn)

Table PoC_SP_UserLogs was created in DB budig-bb-pltsql-05-d


In [15]:
column4 = "Timestamp text"
changeColumns('PoC_SP_Metrics', column4, conn)

Columns Added
Columns changed


In [16]:
logs = getData('PoC_SP_Metrics', 'RecordID',conn)

C:\Users\ramachandran\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [17]:
logs

,103,207,208,209,210,211,212,213,214,215,...,323,324,325,326,327,328,RecordID,pumpID,site,Timestamp
